# Text analysis
## Requisites
Done in python as several libraries are good enough for the text analysis, or even better than in R.
- Data scraping: `twint`
- Data management: `pandas`
  - https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html
- Text processing and sentiment analysis: `nltk`
  - Look at VADER for the sentiment analysis: `nltk.sentiment.vader.SentimentIntensityAnalyzer`
- Plotting: `seaborn` (or the built-in pandas graphs)
- Topic analysis: `corextopic`
  - <a href='https://github.com/gregversteeg/corex_topic'>Github repo</a>

## Steps
1. Scrape tweets
2. Import the data into pandas
3. Sentiment analysis
4. Process the text (e.g. tokenization, lemmatization, corpus, etc.)
5. Word cloud
6. Topic analysis
7. Combine both sentiment and topic analysis

## 1. Scrape tweets
Take into consideration that it can take around 1 hour to compile all of the tweets

In [245]:
import twint

file_name = f"./all_hashtags.csv"

In [246]:
%%script false --no-raise-error
# https://github.com/twintproject/twint/wiki/Configuration

hashtags = [
    "#ABPSouthampton",
    "#DP_World",
    "#GaPorts",
    "#PortofHamburg",
    "#HutchisonPPC",
    "#LondonPortAuth",
    "#PANYNJ",
    "#portdebarcelona",
    "#PortMTL",
    "#Port_Houston",
    "#PortofAntwerp",
    "#felixstowe_port",
    "#portoflongbeach",
    "#PortofLA",
    "#PortofOakland",
    "#PortOfRotterdam",
    "#PortofSeattle",
    "#PortVancouver",
    "#Port_Zeebrugge",
    "#portodigenova",
    "#PuertoAlgeciras",
    "#PuertodeCtg",
    "#SCPorts",
    "#PortofVirginia",
    "#AutPortValencia"
]

c = twint.Config()
c.Output = file_name
for hashtag in hashtags:
    print("test")
    c.Limit = 1000
    c.Store_csv = True
    # c.Search = '"port" and "expansion"'
    # c.Search = '"port"'
    c.Search = hashtag
    c.Filter_retweets = True
    c.Lang = "en"
    twint.run.Search(c)

## 2. Importing the data

In [247]:
import pandas as pd
twitter_data = pd.read_csv(file_name)
twitter_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528 entries, 0 to 527
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               528 non-null    int64  
 1   conversation_id  528 non-null    int64  
 2   created_at       528 non-null    object 
 3   date             528 non-null    object 
 4   time             528 non-null    object 
 5   timezone         528 non-null    int64  
 6   user_id          528 non-null    int64  
 7   username         528 non-null    object 
 8   name             528 non-null    object 
 9   place            8 non-null      object 
 10  tweet            528 non-null    object 
 11  language         528 non-null    object 
 12  mentions         528 non-null    object 
 13  urls             528 non-null    object 
 14  photos           528 non-null    object 
 15  replies_count    528 non-null    int64  
 16  retweets_count   528 non-null    int64  
 17  likes_count     

In [248]:
import re

# Only take the columns that are interesting
reduced_twitter = twitter_data[
    # Only English tweets
    (twitter_data['language']=='en') &
    # Only original tweets, no retweets
    (twitter_data['retweet']==False)
]

# Reduce size by only taking tweets and id
columns_to_keep = pd.Series(['id', 'tweet'])
reduced_twitter = reduced_twitter[columns_to_keep]

# Clean tweets from hastags, mentions, etc.
regex_cleanup_strings = [
    # Remove links
    r"\S*https?:\S*",
    # Remove mentions
    r"(?:\@|https?\://)\S+",
    # Remove hashtags
    "#([a-zA-Z0-9_]{1,50})"
]
reduced_twitter["clean_tweet"] = reduced_twitter["tweet"]
for reg_string in regex_cleanup_strings:
    reduced_twitter["clean_tweet"] = reduced_twitter["clean_tweet"].replace(reg_string, '', regex=True)

reduced_twitter.reset_index(drop=True,inplace=True)
reduced_twitter.head()

id                                              tweet  \
0  1492573501542313990  @arabnews @united4wildlife @EarthshotPrize @Ke...   
1  1492537612757872645  Speaking of #DP_World, they are now collaborat...   
2  1492537610253836296  EarthShot winners and finalists got to promote...   
3  1492492122557145092  @JaniceC28477503 @CamillaTominey Speaking of #...   
4  1492490261347373062  @JaniceC28477503 @CamillaTominey EarthShot win...   

                                         clean_tweet  
0          was wonderful and seeing  do so much w...  
1  Speaking of , they are now collaborating with ...  
2  EarthShot winners and finalists got to promote...  
3    Speaking of , they are now collaborating wit...  
4    EarthShot winners and finalists got to promo...

### 2. Sentiment analysis
For sentiment analysis, VADER rates depending on the form of the word. Therefore, it should NOT be lemmatized or stemmed. For topic classification, it SHOULD be, as a bag-of-words is given.

In [249]:
# Sentiment analysis using VADER
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

sentiment_analysis = pd.DataFrame(sid.polarity_scores(clean_tweet) for clean_tweet in reduced_twitter["clean_tweet"])

processed_twitter = pd.concat([reduced_twitter,  sentiment_analysis], axis=1)
processed_twitter.sort_values(by=['pos'], ascending=False).head(10)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/antoniosanchezmartin/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


id                                              tweet  \
60   1483464759227953153  Beautiful shots! Thanks for sharing! #georgiap...   
298  1471231722654162950        Great photo! #keepingfreightmoving #SCPorts   
280  1489243815261220872  Great photo, @JoeySovine! #keepingfreightmovin...   
6    1492183718102016004  @MayhewCharlie @DP_World @EarthshotPrize @unit...   
11   1484150349032472576  @UNFCCC @EarthlingEco Save earth 🌎 Save Trees ...   
198  1495782691161223172  What a beautiful Monday!#California #LosAngele...   
153  1454860877681856520  The Port Authority PBA wishes all a Happy, Enj...   
127  1475233152285220868  The Port Authority PBA wishes all a Joyous Kwa...   
337  1249614476615331840  #PortofVirginia takes #temperature #checks aft...   
143  1465100469869633536  The Port Authority PBA wishes all who celebrat...   

                                           clean_tweet  neg    neu    pos  \
60             Beautiful shots! Thanks for sharing!     0.0  0.164  0.836   
298                                     Great photo!    0.0  0.185  0.815   
280                                    Great photo,     0.0  0.196  0.804   
6          Yes thank you  for supporting these wond...  0.0  0.250  0.750   
11                        Save earth 🌎 Save Trees 🌲🌲🌲   0.0  0.319  0.681   
198                       What a beautiful Monday!      0.0  0.323  0.677   
153  The Port Authority PBA wishes all a Happy, Enj...  0.0  0.326  0.674   
127  The Port Authority PBA wishes all a Joyous Kwa...  0.0  0.348  0.652   
337                  takes   after  positive case       0.0  0.455  0.545   
143  The Port Authority PBA wishes all who celebrat...  0.0  0.455  0.545   

     compound  
60     0.8802  
298    0.6588  
280    0.6249  
6      0.9325  
11     0.7506  
198    0.6360  
153    0.8860  
127    0.9565  
337    0.5574  
143    0.9617

### 3. Process the text (e.g. tokenization, lemmatization and stop-words)

In [250]:
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
import nltk

test_sentence = "This is a very random test sentence. Let's see what happens churches"
print(test_sentence)

def sentence_processing(sentence:str, printing:bool = False):
    # Tokenize
    tokens = nltk.tokenize.word_tokenize(sentence.lower())
    if printing: print(tokens)

    # Remove punctuation
    tokens = [token for token in tokens if token.isalnum()]
    if printing: print(tokens)

    # Remove stopwords
    stop_words = set(nltk.corpus.stopwords.words('english'))  
    tokens = [w for w in tokens if not w in stop_words]
    if printing: print(tokens)

    # Lemmatice
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    if printing: print(tokens)

    return tokens

sentence_processing(test_sentence, printing=True)

This is a very random test sentence. Let's see what happens churches
['this', 'is', 'a', 'very', 'random', 'test', 'sentence', '.', 'let', "'s", 'see', 'what', 'happens', 'churches']
['this', 'is', 'a', 'very', 'random', 'test', 'sentence', 'let', 'see', 'what', 'happens', 'churches']
['random', 'test', 'sentence', 'let', 'see', 'happens', 'churches']
['random', 'test', 'sentence', 'let', 'see', 'happens', 'church']


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/antoniosanchezmartin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/antoniosanchezmartin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/antoniosanchezmartin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/antoniosanchezmartin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['random', 'test', 'sentence', 'let', 'see', 'happens', 'church']

In [251]:
processed_twitter["tokens"] = processed_twitter["clean_tweet"].apply(lambda sentence: sentence_processing(sentence))
processed_twitter.head()

id                                              tweet  \
0  1492573501542313990  @arabnews @united4wildlife @EarthshotPrize @Ke...   
1  1492537612757872645  Speaking of #DP_World, they are now collaborat...   
2  1492537610253836296  EarthShot winners and finalists got to promote...   
3  1492492122557145092  @JaniceC28477503 @CamillaTominey Speaking of #...   
4  1492490261347373062  @JaniceC28477503 @CamillaTominey EarthShot win...   

                                         clean_tweet    neg    neu    pos  \
0          was wonderful and seeing  do so much w...  0.110  0.439  0.451   
1  Speaking of , they are now collaborating with ...  0.154  0.726  0.120   
2  EarthShot winners and finalists got to promote...  0.000  0.737  0.263   
3    Speaking of , they are now collaborating wit...  0.154  0.726  0.120   
4    EarthShot winners and finalists got to promo...  0.000  0.737  0.263   

   compound                                             tokens  
0    0.9384  [wonderful, seeing, much, innovation, showcase...  
1   -0.2023  [speaking, collaborating, another, charity, sh...  
2    0.6908  [earthshot, winner, finalist, got, promote, wo...  
3   -0.2023  [speaking, collaborating, another, charity, sh...  
4    0.6908  [earthshot, winner, finalist, got, promote, wo...

### Topic categorization

In [252]:
from sklearn.feature_extraction.text import CountVectorizer

# Obtain bag-of-words sparse matrix
corpus = [' '.join(tokens) for tokens in processed_twitter["tokens"]]
print(corpus[0])

vectorizer = CountVectorizer(analyzer='word', stop_words='english')
word_bag = vectorizer.fit_transform(corpus) # this is a sparse matrix!
word_features = vectorizer.get_feature_names_out()
ids = processed_twitter["id"].tolist()
print(word_features)
print(word_bag.shape)
len(word_features)


wonderful seeing much innovation showcase brilliant speech amazing glad dx signed declaration thank
['10' '10th' '11' ... 'ørsted' '𝐓𝐄𝐔' '𝗿𝗲𝗰𝗼𝗿𝗱']
(364, 1650)


1650

Note that `anchor_strength=2` would mean that the anchor words count twice as much as a normal one.

In [253]:
# Anchor words
anchors =[
    ['job', 'economy', 'money', 'income', 'trees'], # economy
    ['earth', 'tree', 'sustainability'], #sustainability
    ['city', 'people', 'neighborhood', 'school'] #people
]

anchors = [sentence_processing(' '.join(set)) for set in anchors] # do the processing so it matches

In [254]:
# Topic modelling
from corextopic import corextopic as ct
from corextopic import vis_topic as vt
import numpy as np
# Train the CorEx topic model
topic_model = ct.Corex(n_hidden=5)  # Define the number of latent (hidden) topics to use.
topic_model.fit(
    word_bag, 
    anchors=anchors,
    anchor_strength=3,
    words=word_features, 
    docs=ids
)

# Get the terms for the topics
topics = topic_model.get_topics()
for topic_n,topic in enumerate(topics):
    # w: word, mi: mutual information, s: sign
    topic = [(w,mi,s) if s > 0 else ('~'+w,mi,s) for w,mi,s in topic]
    # Unpack the info about the topic
    words,mis,signs = zip(*topic)    
    # Print topic
    topic_str = str(topic_n+1)+': '+', '.join(words)
    print(topic_str)

# Check to which cluster each tweet belongs
classification = pd.DataFrame({"Topic": pd.Series(topic_model.labels.tolist())})
processed_twitter = pd.concat([processed_twitter, classification], axis=1)

# Export summary data
vt.vis_rep(topic_model, column_label=word_features, prefix='topic_class_summary')


1: job, director, executive, economy, rest, griff, eternal, north, embrace, lord
2: bay, chassis, milestone, shipment, development, waiting, amp, ship, continuing, ca
3: city, school, people, help, rail, capacity, end, uk, education, good
4: angeles, los, need, click, vessel, congestion, ~virginia, update, instant, understand
5: record, volume, read, port, container, article, long, 2021, beach, oakland
Print topics in text file


In [255]:
# Final dataframe
processed_twitter.head()

id                                              tweet  \
0  1492573501542313990  @arabnews @united4wildlife @EarthshotPrize @Ke...   
1  1492537612757872645  Speaking of #DP_World, they are now collaborat...   
2  1492537610253836296  EarthShot winners and finalists got to promote...   
3  1492492122557145092  @JaniceC28477503 @CamillaTominey Speaking of #...   
4  1492490261347373062  @JaniceC28477503 @CamillaTominey EarthShot win...   

                                         clean_tweet    neg    neu    pos  \
0          was wonderful and seeing  do so much w...  0.110  0.439  0.451   
1  Speaking of , they are now collaborating with ...  0.154  0.726  0.120   
2  EarthShot winners and finalists got to promote...  0.000  0.737  0.263   
3    Speaking of , they are now collaborating wit...  0.154  0.726  0.120   
4    EarthShot winners and finalists got to promo...  0.000  0.737  0.263   

   compound                                             tokens  \
0    0.9384  [wonderful, seeing, much, innovation, showcase...   
1   -0.2023  [speaking, collaborating, another, charity, sh...   
2    0.6908  [earthshot, winner, finalist, got, promote, wo...   
3   -0.2023  [speaking, collaborating, another, charity, sh...   
4    0.6908  [earthshot, winner, finalist, got, promote, wo...   

                                Topic  
0  [False, False, True, False, False]  
1  [False, True, False, False, False]  
2  [False, False, True, False, False]  
3  [False, True, False, False, False]  
4  [False, False, True, False, False]

Quick explanation of the important output files in folder:
- `labels.txt` -> To which group does each tweet belong
  - Similar to `topic_model.labels`
- `topics.txt` -> Top words of each topic

# TODO:
1. Get some nice graphs/plots (e.g. wordclouds from the tokens across all tweets, sentiment analysis summary, etc.)
2. (Maybe) change the "topic" column in the final dataframe for just a single number
   - I.e. instead of `[False True False]` write `2`.
     - Problem -> What about `[False True True`? It belongs to two clusters
3. Finish topic classification with the large dataset (i.e. port_only.csv).
   1. Maybe change the anchors
   2. Get the final topics (i.e. number of topics, keywords, etc.)
   3. Get some nice graphs (e.g. words per topic)
4. (Maybe) fancy cross-correlation plot of sentiment by topic (i.e. are "sustainable" tweets more positive or negative?)
5. Do all of the above for the large dataset (`port_only`), not the small one (`all_hashtags`).